**CẤU TRÚC DỮ LIỆU**

```
[
    [
        "解",
        "giái giải giới", <---Reading: cách nhau bởi SPACE
        "", <---Cùng cột với Reading: cách nhau bởi SPACE
        "",<---Tags: cách nhau bởi SPACE
        [
            "[giải] cởi (áo)",
            "[giải] giải phóng, giải toả",
            "[giải] giảng giải",
            "[giải] giải đi, dẫn đi"
        ],
        {
            "Strokes": "13",
            "Radical": "giác 角 (+6 nét)",
            "PenStrokes": "ノフノフ一一丨フノノ一一丨",
            "Shape": "⿰角⿱刀牛",
            "Unicode": "U+89E3"
        }
    ],
    [...]
    ...
]
-------
{
    "BookNo": "Book01",
    "Page": "No70-119",
    "No": 83,
    "CatGroup": "HN01_03",
    "GroupKJ": "月",
    "GroupHV": "NGUYỆT",
    "GroupVN": "mặt trăng",
    "Kanji": "謂",
    "Typing": "いわく",
    "Hiragana": "iwaku",
    "HanViet": "VỊ",
    "Vietnamese": "gọi là",
    "HintRemember": "bác sỹ NÓI về bệnh DẠ DÀY GỌI LÀ chuẩn đoán"
}
```

In [1]:
### Chạy 2 lần tương ứng với kanji_bank_1.json và kanji_bank_2.json
kanji_bank_IN_file = '../kanji_bank_2.json'
kanji_bank_OUT_file = 'yomitan/kanji_bank_2.json'

### Đường dẫn bên dưới không cần đổi mỗi lần chạy
eng_kanji_bank01_IN_file = '../kanjidic_english/kanji_bank_1.json'
eng_kanji_bank02_IN_file = '../kanjidic_english/kanji_bank_2.json'

tu_tha_dongtu_IN_file = 'TuThaV/output.json'
hacknao_IN_file = "../../MERGE_ALL_KANJI.json"
all_mimikara_IN_file = "ALL_Mimikara (N2 and N3).json"

import json
import wanakana # https://github.com/Starwort/wanakana-py (pip install wanakana-python)

def load_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return json.load(f)

def handle_data_after_fetching(kanji_bank, hacknao_data, tu_tha_data, mimikara_data):
    data_set_hn = {}
    for obj in hacknao_data:
        cat_group = obj.get("CatGroup")
        if cat_group:
            if cat_group in data_set_hn:
                data_set_hn[cat_group]["objList"].append(obj)
            else:
                data_set_hn[cat_group] = {
                    "groupMeta": obj,
                    "objList": [obj]
                }

    for obj_hn in hacknao_data:
        for obj_kanji_bank in kanji_bank:
            if obj_hn.get("Kanji") == obj_kanji_bank[0]:

                kanji_pen_strokes = obj_kanji_bank[5].get('PenStrokes') # Ghi nhớ
                kanji_shape = obj_kanji_bank[5].get('Shape', '') # Hình thái
                kanji_radical = obj_kanji_bank[5].get('Radical') # Bộ
                
                hanviet_nghia = f"{obj_hn.get("HanViet")} ({obj_hn.get("Vietnamese")})"
                no_kanji_id = f"{obj_hn['Kanji']} - {obj_hn['No']}"
                
                ### Modify kanjiBank data based on obj_hn
                # obj_hn.get("HanViet")
                obj_kanji_bank[3] = f"{kanji_pen_strokes} {kanji_shape}"

                obj_kanji_bank[4].insert(0, "---[VN]")
                # obj_kanji_bank[4].insert(0, f"{obj_hn.get('HintRemember')}")
                obj_kanji_bank[4].insert(0, f"{obj_hn['Kanji']} ({obj_hn['Typing']} - {obj_hn['Hiragana']}) [{obj_hn.get('HintRemember')}]")
                obj_kanji_bank[4].insert(0, f"[Bộ {kanji_radical}]")
                obj_kanji_bank[4].insert(0, f"{hanviet_nghia} [{no_kanji_id}]")
                ### ... (modify other fields as needed)
                ### Thêm nghĩa tiếng anh
                eng_kanji = all_eng_kanji_bank_obj[obj_hn['Kanji']]
                if eng_kanji:
                    obj_kanji_bank[4].append("---[EN]")
                    obj_kanji_bank[4] += eng_kanji[4]
                    # Thêm Reading 01
                    temp_ = obj_kanji_bank[1]
                    if temp_:
                        temp_ = f"{temp_} {eng_kanji[1]}"
                    else:
                        temp_ = eng_kanji[1]
                    # Thêm Wana (liền sau) trước khi gán lại
                    obj_kanji_bank[1] = add_wanakana(temp_)
                    # Thêm Reading 02
                    temp_ = obj_kanji_bank[2]
                    if temp_:
                        temp_ = f"{temp_} {eng_kanji[2]}"
                    else:
                        temp_ = eng_kanji[2]
                    # Thêm Wana (liền sau) trước khi gán lại
                    obj_kanji_bank[2] = add_wanakana(temp_)
                ###
                obj_kanji_bank[4].append("---[Group]")
                obj_kanji_bank[4].append(get_family(data_set_hn, obj_hn.get("CatGroup")))
                
                ### Thêm thông tin Tự Tha V
                tu_tha_V_obj = tu_tha_data.get(obj_hn.get("Kanji"))
                if tu_tha_V_obj:
                    obj_kanji_bank[4].append("--------------")
                    obj_kanji_bank[4].append("--[Tự Tha V]--")
                    obj_kanji_bank[4].append(tu_tha_V_obj['TuDongTu'])
                    obj_kanji_bank[4].append(tu_tha_V_obj['ThaDongTu'])
                
                ### Thêm thông tin Mimikara
                mimikara_objs = find_value_by_key_contains_substring(mimikara_data, obj_hn.get("Kanji"))
                if mimikara_objs:
                    obj_kanji_bank[4].append("--------------")
                    obj_kanji_bank[4].append("--[Mimikara]--")
                for index, mimikara_obj in enumerate(mimikara_objs):
                    if mimikara_obj:
                        obj_kanji_bank[4].append(f"[{mimikara_obj['JLPT']}][{mimikara_obj['titleUnit']}][{mimikara_obj['KeyFull']}]")
                        obj_kanji_bank[4].append(f"{mimikara_obj['kanjiFull']} {mimikara_obj['hanviet']} ({mimikara_obj['nghia']})")
                        viduJA = mimikara_obj.get('viduJA')
                        if viduJA: obj_kanji_bank[4].append(viduJA)
                        viduVN = mimikara_obj.get('viduVN')
                        if viduVN: obj_kanji_bank[4].append(viduVN)
                        vidunotfirst = mimikara_obj.get('vidunotfirst')
                        if vidunotfirst: obj_kanji_bank[4].append(vidunotfirst)
                        if index != len(mimikara_objs) - 1 : obj_kanji_bank[4].append("--------------")

def add_wanakana(in_readings):
    if in_readings:
        words = in_readings.split()
        result = []
        for word in words:
            result.append(word + "(" + wanakana.to_romaji(word, uppercase_katakana=True) + ")")
        return " ".join(result)
    else:
        return in_readings

def get_family(data_set_hn, cat_group):
    _tmp = ""
    obj_list = data_set_hn[cat_group]["objList"]
    for obj in obj_list:
        _tmp += f"{obj['Kanji']} ({obj['HanViet']}-{obj['Vietnamese']}) "
    return _tmp

def find_value_by_key_contains_substring(obj, substring):
  results = []
  for key, value in obj.items():
    if substring in key:
      value['KeyFull'] = key
      results.append(value)
  return results

### Gọi các hàm xử lý
all_kanji_bank = load_json(kanji_bank_IN_file)
all_hacknao_data = load_json(hacknao_IN_file)
all_tu_tha_dongtu_data = load_json(tu_tha_dongtu_IN_file)
all_mimikara_data = load_json(all_mimikara_IN_file)

eng_kanji_bank01_data = load_json(eng_kanji_bank01_IN_file)
eng_kanji_bank02_data = load_json(eng_kanji_bank02_IN_file)
all_eng_kanji_bank_data = eng_kanji_bank01_data + eng_kanji_bank02_data
all_eng_kanji_bank_obj = {}
for eng_kanji_item in all_eng_kanji_bank_data:
    all_eng_kanji_bank_obj[eng_kanji_item[0]] = eng_kanji_item

handle_data_after_fetching(all_kanji_bank, all_hacknao_data, all_tu_tha_dongtu_data, all_mimikara_data)

### Phần xử lý kết quả sau khi cập nhật kanji_bank
### Ghi dữ liệu vào file JSON
with open(kanji_bank_OUT_file, 'w', encoding='utf-8') as f:
    json.dump(all_kanji_bank, f, ensure_ascii=False)